In [20]:
from whbase import *
from collections import *

def update_key_value(a, b):
    '''updating key & values (values will be appened for existing key)'''
    c = {}
    if len(a) == 0: return b
    tls = lambda ls : ls if type(ls) is list else [ls]
    for k, v in a.items():
        c[k] = tls(a[k]) + tls(b[k]) if k in list(b) else tls(a[k])
    else: return {**b, **c}

class WhBase:
    
    pane_search = "/html/body/div[1]/div/div/div[3]/div/div[1]/div/div/div[2]/div/div[2]"
    body_chat_search_btn = "//div[@id='main']//div[@class='_3ndVb']"
    body_chat_name = "//div[@id='main']//div[@class='_21nHd']"
    body_text_area = "//footer[@class='_3E8Fg']//p"
    body_msg_nav_arrow = '/html/body/div[1]/div/div/div[4]/div/div[2]/div/div[1]/span/div'
    init_info = "//div[@data-testid='intro-text'][contains(.,'Use WhatsApp on up to 4 linked devices and 1 phone at the same time.')]"
    
    pane_search = "//div[@id='side']//div[@data-testid='chat-list-search']"
    pane_search_back ="//div[@id='side']//span[@data-testid='search']"
    pane_search_cancel = "//div[@id='side']//span[contains(@data-testid,'x-alt')]"
    pane_search_filter = "//div[@id='side']//button[@aria-label='Unread chats filter']"
    
    active_chat_title = "//div[@id='main']//span[@data-testid='conversation-info-header-chat-title']"
    active_chat_search = "//div[@id='main']//span[@data-testid='search-alt']"
    
    msg_menu = '//*[contains(concat( " ", @class, " " ), concat( " ", "_3bcLp", " " ))'

    
    def __init__(self, url=None):
        chrome_options = Options()
        chrome_options.add_experimental_option("debuggerAddress", "127.0.0.1:9222")
        self.url = 'whatsapp.com' if url is None else url
        self.driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
        self.wait = WebDriverWait(self.driver, 5)
        self.ac = ActionChains(self.driver)
        self.read_store = defaultdict()
        self.store_msg = pd.DataFrame([])
        self.pane_data = defaultdict()
        self.chat_data = defaultdict()
        print(self.driver.title)
    
    @property
    def get_driver(self):
        return self.driver
    
    @property
    def reload_get(self):
        wait = WebDriverWait(self.driver, 60)
        self.driver.refresh()
        search_bar_xpt = "/html/body/div[1]/div/div/div[3]/div/div[1]/div/div/div[2]/div/div[2]"
        element = wait.until(EC.presence_of_element_located((By.XPATH, search_bar_xpt)))
        print('reload success, whatsapp ready')
        return self.driver
    
    def xpelem(self, xpt):
        try: return self.wait.until(EC.presence_of_element_located((By.XPATH, xpt)))
        except: return None
    
    def xptext(self, xpt):
        try: return self.driver.find_element(By.XPATH, xpt).text
        except: return None
        
    def xpget(self, xpt):
        try: return self.driver.find_element(By.XPATH, xpt)
        except: return None
    
    def click(self, xpt):
        xp = self.xpelem(xpt) if type(xpt) is str else xpt
        if xp is None: 
            print("xpath not present")
            return None
        try:
            xp.click()
            return 1
        except: return 0
    
    def move_click(self, xpt):
        try: 
            self.ac.move_to_element(self.xpget(xpt)).click(xpt).perform()
            return 1
        except: return 0
        
    def hover_click(self, xp1, xp2):
        try: 
            self.ac.move_to_element(self.xpelem(xp1)).click(self.xpelem(xp2)).perform()
            return 1
        except: return 0
    
    def just_move(self, xp1):
        try:
            self.ac.move_to_element(self.xpget(xp1)).perform()
            return 1
        except: return 0
        
    def paste_text(self, text):
        try: self.ac.send_keys(text).perform()
        except: pass
        
    def click_write_enter(self, xpt, text):
        xp = self.xpelem(xpt) if type(xpt) is str else xpt
        if xp is None: return None
        xp.clear()
        if chr(10) not in text:
            self.ac.move_to_element(xp).click().send_keys(text).send_keys(Keys.RETURN).perform()
        else:
            xx = text.split(chr(10))
            for line in xx:
                self.ac.send_keys(line).perform()
                self.ac.key_down(Keys.SHIFT).key_down(Keys.ENTER).key_up(Keys.SHIFT).key_up(Keys.ENTER).perform()
            self.ac.send_keys(Keys.RETURN).perform()
    
    def sidepane_scroll(self, initial = 1, scrolls=35):
        for i in range(1, scrolls):
            self.driver.execute_script("document.getElementById('pane-side').scrollTop={}".format(initial))
            initial = initial + i + 1
        return initial

    def scroll_by_xpath(self, xpt="(//div[@id='main']//div[@class='ItfyB _3nbHh'])[2]"):
        try:
            xp = self.wait.until(EC.presence_of_element_located((By.XPATH, xpt)))
            self.driver.execute_script("arguments[0].scrollIntoView(true);", xp)
            return 1
        except:
            print('WhBase.scroll_by_xpath(xpt) failed to scroll')
            return 0
        
    def msg_sender_dttm(self, n, base="(//div[@id='main']//div[@data-testid='msg-container'])"):
        xpt = base + "[" + str(n) + "]"
        html = self.wait.until(EC.presence_of_element_located((By.XPATH, xpt))).get_attribute('innerHTML')
        sp = BeautifulSoup(html, "lxml")
        x = sp.find_all('div')
        for i in x:
            try:
                st = i['data-pre-plain-text']
                break
            except:pass
        else: return None, None
        try:
            split_point = st.find(']')
            sender = st[split_point+1:].replace(':','').strip(' ')
            dttm = parse(st.upper()[1:split_point])
            return dttm.strftime("%Y-%m-%d %H:%M"), sender
        except: None, None
            
    def xpbulk(self, xpt):
        try:
            if type(xpt) is str: return self.xptext(xpt)
            elif type(xpt) is list: return [self.xptext(n) for n in xpt]
            elif type(xpt) is dict: return {k : [self.xptext(n) for n in v] if type(v) is list else self.xptext(v) for k, v in xpt.items()}
            else: return None
        except:
            return None
        
    def pane_search_clear(self):
        self.click(self.pane_search_cancel)
        self.click(self.pane_search_back)
        
    def select_chat(self, chat_name):
        if chat_name is not None and chat_name != '':
            x = self.click_write_enter(xpt=self.pane_search, text=chat_name)
            y = self.xptext(self.active_chat_title)
            print('chat body name: ', y)
            if chat_name.lower() in y.lower():
                print('chat selection successful')
                self.pane_search_clear()
                return 1
            else: 0
        else: return None
        
    def send_message(self, chat_names, text):
        cn = [chat_names] if type(chat_names) is str else chat_names
        for n in cn:
            x = self.select_chat(n)
            if x is not None or x !=0:
                self.click_write_enter(text)
                print('message send to ', str(n), ' sucessfully')
                tm.sleep(.5)
            else:
                print('message send failed, chat: ', str(n))
    
    def dict_to_df(self, dc, out=None):
        x = dt.datetime.now().strftime('%b%d%Y_%H%M')
        opt = x  + '.csv' if out is None else x + '_' + out + '.csv'
        df = pd.DataFrame.from_dict(dc).to_csv(opt)
        return df
    
    def chat_msg(self, n, base="(//div[@id='main']//div[@data-testid='msg-container'])", xpt_print=False):
        bi = base + '[' + str(n) + ']'
        if self.xpelem(bi) is None: return None
        else:
            self.move_click(bi + "//div[@role='button'][contains(.,'Read more')]")
            dttm, sender = self.msg_sender_dttm(n)
            dc = {'sender' : [sender],
                  'text' : [self.xptext(bi + "//div[@class='_21Ahp']/span[1]/span")],
                  'q_sender' : [self.xptext(bi + "//div[@class='_3pMOs yKTUI']//div[1]/span")],
                  'q_text' : [self.xptext(bi + "//div[@class='_3pMOs yKTUI']//div[2]/span")],
                  'datetime' : [dttm]}
            return dc
   
    def side_pane(self, n=2, base="(//div[@id='pane-side']//div[@class='_8nE1Y'])"):
        bi = base + '[' + str(n) + ']'
        if self.xpelem(bi) is None: return None
        dc = {'chat_name' : bi + "//div[@class='_21S-L']//span[@dir='auto']",
              'last_text' : bi + "//span[@data-testid='last-msg-status']",
              'last_sender' : [bi + "//div[@class='vQ0w7']//span[@dir='auto']", 
                               bi + "//span[@data-testid='status-dblcheck']"],
              'last_msg_time' : bi + "/div[@class='y_sn4']/div[@class='Dvjym']",
              'new_msg_notif' : bi + "//div[@class='Dvjym']//span[@data-testid='icon-unread-count']"}
        for k, v in dc.items():
            v = [v] if type(v) is str else v
            dc[k] = [self.xptext(j) for j in v]
        return dc

In [5]:
class WA(WhBase):
    
    def __init__(self):
        super().__init__()
        
    def xe(self, xpt, click=False, text=False):
        try:
            x = self.wait.until(EC.presence_of_element_located((By.XPATH, xpt)))
            if click == True: x.click()
            if text == False: return x
            else: return x.text
        except: return None
        
    def xes(self, xpt):
        try: return self.driver.find_elements(By.XPATH, xpt)
        except: return None
        
    def read_unread(self, chatname, match_action:dict):
        chat_name_notif = "//div[@class='_21S-L']//span[@dir='auto' and contains(text()," + chatname + \
        ")]//ancestor::div[@class='_8nE1Y']//div[@class='Dvjym']//span[@data-testid='icon-unread-count']"

    def send(self):
        pass

    def get_unread_chats(self):
        ls = "//div[@id='pane-side']//div[@class='_21S-L']//span[@dir='auto']"

    def search(self):
        pass

    def forward(self, chatname, match_func, forward_to, from_no_of_message=20, from_lastest=True):
        pass

    def reply(self, chatname, match_func, reply_text, from_no_of_message=20, reading_lastest=True):
        pass

    def delete(self, chatname, match_func, from_no_of_message=20, from_lastest=True, delete_for_all=True):
        pass

    def reaction(self, chatname, match_func, from_no_of_message=20, from_lastest=True):
        pass

    def get_msg(self, chatname, no_of_msg=50, msg_date=None):
        pass

    def monitor_contacts(self, list_of_chats, interval=5):
        pass

    def monitor_chats(self, list_of_chats, interval=5):
        pass

    def monitor_cotent(self, list_of_chats, interval=5):
        pass
    
    def xptext_dic(self, dc:dict):
        for k, v in dc.copy().items():
            dc[k] = [self.xe(n, text=True) for n in v] if type(v) is list else self.xe(v, text=True)
        else: return dc
     


In [15]:
x = WA()
d = x.driver

(4) WhatsApp


In [19]:
def forward(d, x, fwto='onami', msgxpt="(//div[@id='main']//div[@data-testid='msg-container'])[last()]")
    a1 = x.xpelem(msgxpt)
    ActionChains(d).move_to_element(a1).perform()
    x.click(x.xpelem("(//div[@data-testid='msg-container'])[last()]//span[@data-testid='down-context']"))
    tm.sleep(1)
    x.click("//ul//li[contains(.,'Forward')]")
    tm.sleep(1)
    x.click("//div[@class='_2IUvV']//span[@data-testid='forward']")
    xpel  = x.xpelem("(//div[contains(@role,'textbox')])[1]")
    ActionChains(d).move_to_element(xpel).click().perform()
    tm.sleep(1)
    ActionChains(d).send_keys_to_element(xpel, fwto).perform()
    tm.sleep(2)
    x.click("(//div[@class='_199zF _3j691 _2N1Gm'])[1]")
    tm.sleep(1)
    x.click("//span[@data-testid='send']")

1

In [18]:
x.click("//span[@data-testid='send']")

1

In [ ]:
tm.sleep(2)
x.click("(//div[@class='_199zF _3j691 _2N1Gm'])[1]")
tm.sleep(2)
x.click("//*[@data-testid='send']")

In [6]:
msgdiv="(//div[@id='main']//div[@data-testid='msg-container'])[last()]"
acc = ActionChains(d)
msgtime, msgarrow = msgdiv + "//div[@class='_2_-To']", msgdiv + "//span[@data-testid='down-context']"
ActionChains(d).move_to_element(x.xpelem(msgtime)).perform()
ActionChains(d).click(x.xpelem(msgarrow)).perform()


In [ ]:
x.click("//div[@class='_2sDI2']//div[contains(text(),'Forward')]")
tm.sleep(.5)
x.click("//div[@class='_2IUvV']//span[@data-testid='forward']")
xx = "(//div[contains(@role,'textbox')])[1]"
ActionChains(d).send_keys_to_element(xx, 'sakib').perform()
x.click("(//div[@class='_199zF _3j691 _2N1Gm'])[1]")
x.click("//*[@data-testid='send']")

In [ ]:
        tm.sleep(.5)
        if self.click("//div[@class='_2sDI2']//div[contains(text(),'Forward')]") == 1:
            time.sleep(.5)
            if self.click("//div[@class='_2IUvV']//span[@data-testid='forward']") == 1:
                time.sleep(.5)
                xpel  = self.xpelem("(//div[contains(@role,'textbox')])[1]")
                acc.move_to_element(xpel).click().send_keys_to_element(xpel, forward_to).perform()
                tm.sleep(1)
                if self.click("(//div[@class='_199zF _3j691 _2N1Gm'])[1]") == 1:
                    tm.sleep(1)
                    if self.click("//*[@data-testid='send']") == 1:
                        print('forward success')
                else: print('cound not select')
            else: print('forward button from foter not found')
        else:  print('msg additional menu not found')

In [ ]:
xpt_by_chatname = lambda chatname: "//div[@class='_21S-L']//span[@dir='auto' and contains(text(),'" + chatname + "')]//ancestor::div[@class='_8nE1Y']"

def pane_xpt(index=None, parent = "(//div[@id='pane-side']//div[@class='_8nE1Y'])", chatname=None):
    if index is not None and parent is not None:
        bi = parent + "[" + str(index) + "]"
    elif parent is None and chatname is not None:
        bi = "//div[@id='pane-side']//div[@class='_21S-L']//span[@dir='auto' and contains(text(),'" + chatname + "')]//ancestor::" + parent
    else:
        bi = parent
    dc = {'chat_name' : bi + "//div[@class='_21S-L']//span[@dir='auto']",
          'last_text' : bi + "//span[@data-testid='last-msg-status']",
          'last_sender' : bi + "//div[@class='vQ0w7']//span[@dir='auto']",
          'last_msg_time' : bi + "/div[@class='y_sn4']/div[@class='Dvjym']",
          'new_msg_notif' : bi + "//div[@class='Dvjym']//span[@data-testid='icon-unread-count']"}
    return dc

def pane_chat_info(store_dc = defaultdict(), bs="//div[@id='pane-side']//div[@class='_21S-L']//span[@dir='auto']"):
    'fetch available chats info from pane without scroll'
    lschat = y.xes(bs)
    for i in lschat:
        bychat = xpt_by_chatname(i.text)
        dc = pane_xpt(parent=bychat)
        for k, v in dc.copy().items():
            q = y.xptext(v)
            dc[k] = q if q is not None else ''
        store_dc = update_key_value(store_dc, dc)
    else: return y.dict_to_df(store_dc)
    
def get_unread_msg(chatname='Emergency SOC Group',store_dc = defaultdict(),msgbase="(//div[@id='main']//div[@data-testid='msg-container'])"):
    'fetch unread available msg inside group without scroll'
    chatxpt = xpt_by_chatname(chatname)
    dc = pane_xpt(parent=chatxpt)
    print('chatname: ', chatname, 'unread_sms: ',  y.xptext(dc['new_msg_notif']))
    if y.xptext("//div[@id='main']//span[@data-testid='conversation-info-header-chat-title']") not in chatname:
        y.click(chatxpt)
    avmsg = y.xes(msgbase)
    for n in range(len(avmsg)):
        dc = y.chat_msg('last()-' + str(n))
        store_dc = update_key_value(store_dc, dc)
    else:
        df = y.dict_to_df(store_dc, chatname)
        return store_dc, df
    
'''
//*[contains(@class,'head')]
//*[contains(text(),'abcd')]
//*[contains(normalize-space(text()),'abcd')]
//div[contains(@class, 'measure-tab') and contains(.//span, 'someText')]
//div[contains(@class, 'measure-tab') and contains(.,'someText')]
//div[@class='credit_summary_item' and contains(., 'Professor')]
//div[@class='credit_summary_item' and text()='Professor']
//*[text() = 'qwerty' and not(text()[2])]
//*[starts-with(@class,'head')] 
//*[ends-with(@class,"head")]
//*[starts-with(name(), 'him')]

//button[text()='Submit']
//button[normalize-space(text())='Submit']
//button/text()
//table[count(tr)=1]
//ol/li[position()=2]
//form[@id='myform']//input[@type='submit']
'''

In [ ]:
(//*[contains(normalize-space(text()) , 'Update of Site Down')]/ancestor::div[@data-testid='msg-container'])[last()]
(//*[contains(normalize-space(text()) , 'Update of Site Down') and contains(., 'Region: 2G/3G/4G')]/ancestor::div[@data-testid='msg-container'])[last()]

In [ ]:
contain_text = lambda value: "contains(normalize-space( text() ) , '" + value + "')"
contain_dot = lambda value: "contains(., '" + value + "')"
start_with = lambda value: "starts-with(normalize-space( text() ), '" + value + "')"

In [ ]:
match = lambda value: "matches(normalize-space( text() ) , '" + value + "')"

In [ ]:
(//*[matches(normalize-space(text()) , 'Update of Site Down') and matches(., 'Region: 2G/3G/4G')]/ancestor::div[@data-testid='msg-container'])[last()]

In [ ]:
match_criteria = {'msg_contains' = ['Update of Site Down at','Region: 2G/3G/4G'], 'sender':[], 'time':[]}
action = {'func':'forward_to','receipient':['onami']}

In [ ]:
"//div[contains(concat(' ',normalize-space(@class),' '),' foobar ')]""
"//*[contains(text(),'Power shutdown ongoing') and contains(.,'3:56 pm')]"

In [ ]:
def xpgen(tag, attr, value):
    tag = tag if tag is not None else "*"
    attr = attr if attr is not None else "."
    value = value if tag is not None else "," + str(value)
    contain = tag + "[ contains(@"+ attr + ",'" + str(value) + "') ]"
    print(contain)
xpgen('div', 'class', 'i am')

In [ ]:
tag + "[text()=" + value + "]"
tag = "[starts-with(name(), " + value & "]"
tag + "[@id='" + value + "' and position()=" + str(2) + ']'

"div[(x and y) or not(z)]'
"div[@class="head"][@id="top"]"
"section[.//h1[@id='hi']]"



count()                    # //table[count(tr)=1]
position()                 # //ol/li[position()=2]

//section[.//h1[@id='hi']]  #This returns <section> if it has an <h1> descendant with id='hi'.

In [ ]:
y = WA()
d = y.get_driver

In [ ]:
#indexed_chat_info(y.xes("(//div[@id='pane-side']//div[@class='_8nE1Y'])"))

In [ ]:
#d = get_chat_info()

In [ ]:
(//*[starts-with(text(),'Update of Site Down')])not(//*[contains(.,'Hosna Ara')])

In [ ]:
//*[contains(normalize-space(text()) , 'Hasib')]
//*[contains(normalize-space(text()) , 'Update of Site Down')]

In [ ]:
msg = get_unread('Robi & ABHT')

In [ ]:
for q in msg:
    m, n = q[0], q[1]
    print(type(m['text']), n)


In [ ]:
chatname = 'Emergency SOC Group'
msglist = get_unread(chatname)
print(len(msglist))
if y.xptext("//div[@id='main']//span[@data-testid='conversation-info-header-chat-title']") in chatname:
    print('already selected correct chat', chatname)

In [ ]:
for i in msglist:
    print(i.text)


In [ ]:
ls = 
for i in ls:
    bychat = xpt_by_chatname(i.text)
    dc = pane_xpt(parent=bychat)
    for k, v in dc.copy().items():
        q = y.xptext(v)
        dc[k] = q if q is not None else ''
    print(dc)
    print('-------------')


In [ ]:
#chat_name_notif = "//div[@class='_21S-L']//span[@dir='auto' and contains(text()," + chatname + \
#")]//ancestor::div[@class='_8nE1Y']//div[@class='Dvjym']//span[@data-testid='icon-unread-count']"

In [ ]:
unread_only=True, match_str= [], forward=['onami','omi'], start=None, end=None

In [ ]:
d.find_element(By.XPATH, "(//div[@id='pane-side']//div[@class='_8nE1Y'])")

In [ ]:
xe = lambda xpt: d.find_elements(By.XPATH, xpt)

In [ ]:
print(x)

In [ ]:
ls = "//div[@id='pane-side']//div[@class='_21S-L']//span[@dir='auto']"

In [ ]:
match_action_1 = {'msg_contains': ['Update of Site Down at','Region: 2G/3G/4G'], 
                'from_contact':[], 
                'action_func' :'forward_msg',
                'action_func_arg':{'forward_to':['Robi & e.co Ops','OPS & RO']}}

match_action_2 = {'msg_contains': ['ICMP Delay & Packet Loss Update'], 
                'from_contact':[], 
                'action_func' :'forward_msg',
                'action_func_arg':{'forward_to':['operations']}}

match_action_3 = {'msg_contains': ['Network Update','CTO and CEBO traces','NCEIP QoS related issue'], 
                'from_contact':[], 
                'action_func' :'forward_msg',
                'action_func_arg':{'forward_to':['SM & CN_Monitoring']}}

m_action = [match_action_1, match_action_2, match_action_3]

In [ ]:
x.select_chat('emergency')

In [ ]:
x.chat_msg('last()')

In [ ]:
e = x.xpget("(//div[@id='main']//div[@data-testid='msg-container'])")

In [ ]:
y = d.find_elements(By.XPATH, "(//div[@id='main']//div[@data-testid='msg-container'])")

In [ ]:
class WaTem(WhBase):

    def duplicate_none_chk(self, sdic, dc):
        dpcount, nonecount = 0, 0
        for k, v in dc.items():
            if v is None: nonecount = nonecount + 1
            else:
                if v in list(sdic.values()): 
                    dpcount = dpcount + 1
        else:
            if nonecount >=3: none_flag = True
            else: none_flag = False

            if dpcount>=3: dp_flag = True
            else: dp_flag = False
            return none_flag, dp_flag


    def watem_iter(self, method, start=1, end=50, step=1, sdict=defaultdict()):
        dc_is_none = 0
        for n in range(start, end, step):
            dc = getattr(self, method)(n)
            if dc is None and dc_is_none>3:
                print('return caller sdict & n')
                return sdict, n
            elif dc is not None:
                if len(sdict) == 0: sdict = dc
                else:
                    nf, dpf = self.duplicate_none_chk(sdict, dc)
                    #print(nf, dpf, n, dc)
                    if nf == False and dpf == False:
                        sdict = update_key_value(sdict, dc)
                        dc_is_none = 0
                        yield dc, n
                    else:
                        dc_is_none = dc_is_none + 1
            else:
                dc_is_none = dc_is_none + 1
                print('dc is None', dc_is_none)
        else:
            return sdict, n

In [ ]:
x = WaTem()

def read_msg_action(chatname='emergency', no_of_sms=50, action_dict={'text_match':'update of site down', 'forward':'halim vai'}):
    loop_done = 0
    x.select_chat(chatname)
    while comloop<=no_of_sms:
        dd = x.watem_iter('chat_msg', start=1, end=no_of_sms, step=1, sdict=defaultdict())
        for dc, loop_done in zip(dd):
            print(loop_done)

In [ ]:
class WATEM:

    def tyEx_fn(fn, *arg, **kwarg):
        try: return fn(*arg, **kwarg)
        except: return None

    def tyEx_exp(cls, method, *arg, **kwarg):
        try: return getattr(cls, method)(*arg, **kwarg)
        except: return None



In [ ]:
class WaBase:
    
        
    def accessibility(self, lsxp):
        checklist = ['title', 'pane search box', 'side-pane', 'active chat name', 'chat_write_box']

In [ ]:
class A:
    
    def __init__(self):
        pass
    
    def adder(self, a,  b, c = None):
        if c is None:
            print(a+b)
            return a+b
        else:
            return a+b+c
        
def adder(a,  b, c = None):
    if c is None:
        return a+b
    else:
        print(a+b+c)
        return a+b+c
    
def addx(a, b):
    print(a+b)
            

In [ ]:
x = A()
print(tyEx_exp(x, 'adder', 1, 2))